In [15]:
import pandas as pd
import requests
from datetime import datetime
import tzlocal  # $ pip install tzlocal
import pickle

<h1> Download the station feed </h1>

In [16]:
# Download the feed
url = 'https://gbfs.bcycle.com/bcycle_indego/station_status.json'
report = requests.get(url).json()
report.keys()

dict_keys(['last_updated', 'ttl', 'data'])

In [17]:
# Make a data frame from the report and recover the time of the report
df = pd.DataFrame(report['data']['stations'])
unix_timestamp = df['last_reported'].value_counts().index.values[0]
local_timezone = tzlocal.get_localzone() # get pytz timezone
local_time = datetime.fromtimestamp(unix_timestamp, local_timezone)
local_time

datetime.datetime(2019, 2, 25, 8, 0, 47, tzinfo=<DstTzInfo 'America/New_York' EST-1 day, 19:00:00 STD>)

In [18]:
# Extract general information about stations. Connect it with the list of active stations (eliminate inactive) 
df['Station ID'] = df['station_id'].apply(lambda x: int(x[-4:]))
df['station_capacity'] = df['num_bikes_available']+df['num_docks_available']
stations_df = pd.read_csv('data/indego-stations-2019-01-04.csv').reset_index()
stations_merged = pd.merge(stations_df, df, on = ['Station ID'])
stations_merged = stations_merged[stations_merged['Status']!="Inactive"]

In [19]:
# Load the saved list of most valuable stations. 
# Make a new dataframe that will have the info about station capacity and current state. 
this_filename = 'application/flask_app/most_valuable_stations_dict.pckl'
mv_stations = pickle.load(open(this_filename, 'rb'))
df_mv = pd.DataFrame.from_dict(mv_stations,orient = 'index')
df_mv['station_id'] = df_mv.index
df_mv['station_name'] = df_mv[0]
df_mv = df_mv.drop(columns = 0).reset_index() 
capacity = []
bikes_now = []
st_list = list(mv_stations.keys()); 
for station in st_list: 
    capacity.append(stations_merged[stations_merged['Station ID'] == station]['station_capacity'].values[0])
    bikes_now.append(stations_merged[stations_merged['Station ID'] == station]['num_bikes_available'].values[0])
df_mv['station_capacity'] = capacity
df_mv['n_bikes'] = bikes_now

In [20]:
# write the current state into flask
f = open('application/flask_app/curent_bike_state_mvs.pckl', 'wb')
pickle.dump(df_mv, f)
f.close()

In [21]:
df_mv

,index,station_id,station_name,station_capacity,n_bikes
0,3010,3010,15th & Spruce,19,5
1,3021,3021,18th & JFK,34,18
2,3054,3054,Rodin Museum,35,11
3,3045,3045,13th & Locust,15,6
4,3057,3057,Philadelphia Museum of Art,23,2
5,3020,3020,University City Station,35,22
6,3032,3032,23rd & South,16,7
7,3108,3108,15th & Market,21,10
8,3012,3012,21st & Catharine,27,5
9,3101,3101,11th & South,18,13
